In [1]:
import os
import sys
import random
import shutil
import re
import time
import numpy as np
import glob
import tqdm
import cv2
import pandas as pd
import pickle
import json
from timeit import default_timer as timer

#plot
import matplotlib
import matplotlib.pyplot as plt
ROOT_DIR = os.path.abspath("../../")

In [ ]:
#function that quickly compute IoU and fct that says if a mask is included in another mask
#finally fct that remove useless masks on an image (either if IoU>0.7 or if one included in another:remove the smallest one)
#Note that we will compare bbox and not mask, as it will be faster and enough accurate

In [ ]:
PACKAGE_PARENT = '../../..'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))

from UTILS.utils import plot_bboxes, remove_embedded_bbox

## Load images & masks

In [3]:
path2data = os.path.join(ROOT_DIR, 'UTILS', 'mask&image', 'data')

In [ ]:
#available data
glob.glob(path2data)

In [ ]:
#choose data to work on
img_id = ''

In [ ]:
#load mask
r = pickle.load(open(os.path.join(path2data, img_id[:-4]+'_maskrcnnoutput.pkl'), 'rb'))

In [ ]:
#load image
image = cv2.imread(open(os.path.join(path2data, img_id))

#### processing

In [ ]:
li_bboxes = r['rois'] #bbox: x1,y1,x2,y2
li_masks = r['masks'] #binary masks

In [ ]:
#takes a list of bbox ([(x,y,w,h),...]) with the associated image and plot on the image
plot_bboxes(li_bboxes, image)

## Study IoU

In [ ]:
#x,y,w,h while maskrcnn output x1,y1,x2,y2
binary_masks = res[0]['masks']    
li_bboxes = [(b[1], b[0], b[3]-b[1], b[2]-b[0]) for b in res[0]['rois']] #x,y,w,h while maskrcnn output x1,y1,x2,y2

#### functions

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [ ]:
def iou_metric_batch2(y_trues_in, y_preds_in):
    y_trues_in=(y_trues_in>0).squeeze()
    y_preds_in=(y_preds_in).squeeze()
    unions = np.sum(y_preds_in|y_trues_in,(1,2))
    intersections=np.sum(y_preds_in*y_trues_in,(1,2))
    ious=np.maximum(0.0,np.ceil(20*intersections/np.where(unions==0.0,1,unions)-10)/10)
    return np.mean(np.where(unions==0,1,ious))

#### compare time

In [ ]:
start = timer()
# ...
end = timer()
print(end - start) 

## Study mask in other mask

In [ ]:
#remove from a list of rectangles (tuples: (x,y,w,h)), the rectangle embedded in another one
#note that the (0,0) point in an image is up left.
remove_embedded_bbox(li_bbox, plot_bbox=0)

#### functions

#### compare time

## Final masks post-processing